In [127]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd


def get_jobs(num_jobs, verbose, path, slp_time, p_num):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    
    url = 'https://www.glassdoor.com/Job/machine-learning-engineer-jobs-SRCH_KO0,25_IP{}.htm'.format(p_num)
    #url = 'https://www.glassdoor.com/Job/data-engineer-jobs-SRCH_KO0,13_IP{}.htm'.format(p_num)
    #url = 'https://www.glassdoor.com/Job/data-analyst-jobs-SRCH_KO0,12_IP{}.htm?sortBy=date_desc'.format(p_num)
    #url = 'https://www.glassdoor.com/Job/data-scientist-jobs-SRCH_KO0,14_IP{}.htm'.format(p_num)
    #url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs
            
            
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.v

In [10]:
#path = "C:/Users/Ken/Documents/ds_salary_proj/chromedriver"
path = "/Users/flationschool/Course_Materials/nyc-mhtn-ds-010620-lectures/Final_Project/chromedriver"

df = get_jobs(10, False, path, 15)

#df.to_csv('glassdoor_jobs.csv', index = False)

 x out failed
Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


In [17]:
df2 = get_jobs(10, False, path, 20)

 x out failed
Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


In [22]:
df3 = get_jobs(5, False, path, 20, 21)

 x out worked
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [24]:
df4 = get_jobs(5, False, path, 20, 22)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [25]:
df5 = get_jobs(10, False, path, 20, 23)

 x out failed
Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


In [26]:
df6 = get_jobs(10, False, path, 20, 24)

 x out failed
Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


In [29]:
df7 = get_jobs(5, False, path, 20, 25)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [32]:
df8 = get_jobs(5, False, path, 10, 26)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [35]:
df9 = get_jobs(5, False, path, 10, 27)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [38]:
df10 = get_jobs(5, False, path, 10, 28)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [39]:
df11 = get_jobs(10, False, path, 10, 29)

 x out failed
Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


In [40]:
df12 = get_jobs(25, False, path, 10, 29)

 x out failed
Progress: 0/25
Progress: 1/25
Progress: 2/25
Progress: 3/25
Progress: 4/25
Progress: 5/25
Progress: 6/25
Progress: 7/25
Progress: 8/25
Progress: 9/25
Progress: 10/25
Progress: 11/25
Progress: 12/25
Progress: 13/25
Progress: 14/25
Progress: 15/25
Progress: 16/25
Progress: 17/25
Progress: 18/25
Progress: 19/25
Progress: 20/25
Progress: 21/25
Progress: 22/25
Progress: 23/25
Progress: 24/25
Progress: 25/25


In [44]:
df13 = get_jobs(5, False, path, 10, 30)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [48]:
df14 = get_jobs(7, False, path, 10)

 x out worked
Progress: 0/7
Progress: 1/7
Progress: 2/7
Progress: 3/7
Progress: 4/7
Progress: 5/7
Progress: 6/7
Progress: 7/7


In [52]:
df15 = get_jobs(5, False, path, 10, 3)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [53]:
df16 = get_jobs(5, False, path, 10, 4)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [54]:
df17 = get_jobs(5, False, path, 10, 5)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [55]:
df18 = get_jobs(5, False, path, 10, 6)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [56]:
df19 = get_jobs(5, False, path, 10, 7)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [57]:
df20 = get_jobs(5, False, path, 10, 8)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [60]:
df21 = get_jobs(5, False, path, 10, 9)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [61]:
df22 = get_jobs(5, False, path, 10, 10)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [62]:
df23 = get_jobs(5, False, path, 10, 11)

 x out worked
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [63]:
df24 = get_jobs(5, False, path, 10, 12)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [64]:
df25 = get_jobs(5, False, path, 10, 13)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [65]:
df26 = get_jobs(5, False, path, 10, 14)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [66]:
df27 = get_jobs(5, False, path, 10, 15)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [67]:
df28 = get_jobs(5, False, path, 10, 16)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [68]:
df29 = get_jobs(5, False, path, 10, 17)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [69]:
df30 = get_jobs(5, False, path, 10, 18)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [70]:
df31 = get_jobs(5, False, path, 10, 19)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [71]:
df32 = get_jobs(5, False, path, 10, 20)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [72]:
df33 = get_jobs(5, False, path, 10, 21)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [73]:
df34 = get_jobs(5, False, path, 10, 22)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [75]:
df36 = get_jobs(5, False, path, 10, 24)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [76]:
df37 = get_jobs(5, False, path, 10, 25)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [77]:
df38 = get_jobs(5, False, path, 10, 26)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [78]:
df39 = get_jobs(5, False, path, 10, 27)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [79]:
df40 = get_jobs(5, False, path, 10, 28)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [80]:
df41 = get_jobs(5, False, path, 10, 29)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [81]:
df42 = get_jobs(5, False, path, 10, 30)

 x out worked
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [122]:
finaldf = pd.concat([df,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20], ignore_index = True)

In [123]:
finaldf = pd.concat([finaldf,df21,df22,df23,df24,df25,df26,df27,df28,df29,df30,df31,df32,df33,df34,df36,df37,df38,df39,df40,df41,df42], ignore_index =True)

In [124]:
finaldf

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Senior Data Scientist,$105K-$167K (Glassdoor est.),"About Us\n\nAt GutCheck, we pioneered agile ma...",3.8,GutCheck\n3.8,"Denver, CO","Denver, CO",51 to 200 employees,2009,Company - Private,Advertising & Marketing,Business Services,$10 to $25 million (USD),"Nielsen, Zappi, SurveyMonkey"
1,Data Scientist,$105K-$167K (Glassdoor est.),Data Scientist\n\nAs Data Scientist at HP Inc....,4.0,HP Inc.\n4.0,"San Diego, CA","Palo Alto, CA",10000+ employees,1939,Company - Public,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
2,"Scientist/ Sr. Engineer I, Process Development",$105K-$167K (Glassdoor est.),The Role:\nCurious about HIGH-THROUGHPUT PROCE...,3.4,Moderna Therapeutics\n3.4,"Norwood, MA","Cambridge, MA",501 to 1000 employees,2011,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$1 to $5 million (USD),-1
3,Analytics Manager,$105K-$167K (Glassdoor est.),Responsible for both high level and granular m...,4.4,OneMagnify\n4.4,"Dearborn, MI","Detroit, MI",201 to 500 employees,1967,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1
4,Computational Chemist/Data Scientist,$105K-$167K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310562\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,Data Analyst,$49K-$65K (Glassdoor est.),KellyMitchell matches the best IT and business...,3.6,Kelly Mitchell Group\n3.6,"New Albany, OH","Saint Louis, MO",1001 to 5000 employees,1998,Company - Private,Staffing & Outsourcing,Business Services,$50 to $100 million (USD),-1
248,Data Analyst,$49K-$65K (Glassdoor est.),POSITION TITLE: Data Analyst\n\nREPORTS TO: AV...,3.7,State Employee Credit Union\n3.7,"Santa Fe, NM","Raleigh, NC",1001 to 5000 employees,1937,Nonprofit Organization,Investment Banking & Asset Management,Finance,$500 million to $1 billion (USD),-1
249,Data Analyst,$49K-$65K (Glassdoor est.),"At ALLERE GROUP, we pride ourselves in creatin...",5.0,Allere\n5.0,"Des Moines, IA","Kalamazoo, MI",1 to 50 employees,-1,Company - Private,-1,-1,Less than $1 million (USD),-1
250,Business Data Analyst I,$49K-$65K (Glassdoor est.),The Business Data Analyst I is responsible for...,3.8,Digital Turbine\n3.8,"Austin, TX","Austin, TX",51 to 200 employees,2011,Company - Public,Computer Hardware & Software,Information Technology,$100 to $500 million (USD),-1


In [88]:
df43 = get_jobs(7, False, path, 10, 2)

 x out failed
Progress: 0/7
Progress: 1/7
Progress: 2/7
Progress: 3/7
Progress: 4/7
Progress: 5/7
Progress: 6/7
Progress: 7/7


In [89]:
df44 = get_jobs(7, False, path, 10, 3)

 x out failed
Progress: 0/7
Progress: 1/7
Progress: 2/7
Progress: 3/7
Progress: 4/7
Progress: 5/7
Progress: 6/7
Progress: 7/7


In [90]:
df45 = get_jobs(7, False, path, 10, 4)

 x out failed
Progress: 0/7
Progress: 1/7
Progress: 2/7
Progress: 3/7
Progress: 4/7
Progress: 5/7
Progress: 6/7
Progress: 7/7


In [91]:
df46 = get_jobs(7, False, path, 10, 5)

 x out failed
Progress: 0/7
Progress: 1/7
Progress: 2/7
Progress: 3/7
Progress: 4/7
Progress: 5/7
Progress: 6/7
Progress: 7/7


In [93]:
df47 = get_jobs(5, False, path, 10, 6)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [94]:
df48 = get_jobs(5, False, path, 10, 7)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [95]:
df49 = get_jobs(5, False, path, 10, 8)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [96]:
df50 = get_jobs(5, False, path, 10, 9)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [97]:
df51 = get_jobs(5, False, path, 10, 10)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [98]:
df52 = get_jobs(5, False, path, 10, 11)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [99]:
df53 = get_jobs(5, False, path, 10, 12)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [100]:
df54 = get_jobs(5, False, path, 10, 13)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [101]:
df55 = get_jobs(5, False, path, 10, 14)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [102]:
df56 = get_jobs(5, False, path, 10, 15)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [103]:
df57 = get_jobs(5, False, path, 10, 16)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [104]:
df58 = get_jobs(5, False, path, 10, 17)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [105]:
df59 = get_jobs(5, False, path, 10, 18)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [106]:
df60 = get_jobs(5, False, path, 10, 19)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [107]:
df61 = get_jobs(5, False, path, 10, 20)

 x out worked
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [108]:
df62 = get_jobs(5, False, path, 10, 21)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [110]:
df63 = get_jobs(5, False, path, 10, 22)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [112]:
df64 = get_jobs(5, False, path, 10, 23)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [113]:
df65 = get_jobs(5, False, path, 10, 24)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [114]:
df66 = get_jobs(5, False, path, 10, 25)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [115]:
df67 = get_jobs(5, False, path, 10, 26)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [116]:
df68 = get_jobs(5, False, path, 10, 27)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [117]:
df69 = get_jobs(5, False, path, 10, 28)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [118]:
df70 = get_jobs(5, False, path, 10, 29)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [119]:
df71 = get_jobs(5, False, path, 10, 30)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [125]:
finaldf = pd.concat([finaldf,df43,df44,df45,df46,df47,df48,df49,df50,df51,df52,df53,df54,df55,df56,df57,df58,df59,df60,df61,df62,df63,df64,df65,df66,df67,df68,df69,df70,df71], ignore_index = True)

In [126]:
finaldf

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Senior Data Scientist,$105K-$167K (Glassdoor est.),"About Us\n\nAt GutCheck, we pioneered agile ma...",3.8,GutCheck\n3.8,"Denver, CO","Denver, CO",51 to 200 employees,2009,Company - Private,Advertising & Marketing,Business Services,$10 to $25 million (USD),"Nielsen, Zappi, SurveyMonkey"
1,Data Scientist,$105K-$167K (Glassdoor est.),Data Scientist\n\nAs Data Scientist at HP Inc....,4.0,HP Inc.\n4.0,"San Diego, CA","Palo Alto, CA",10000+ employees,1939,Company - Public,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
2,"Scientist/ Sr. Engineer I, Process Development",$105K-$167K (Glassdoor est.),The Role:\nCurious about HIGH-THROUGHPUT PROCE...,3.4,Moderna Therapeutics\n3.4,"Norwood, MA","Cambridge, MA",501 to 1000 employees,2011,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$1 to $5 million (USD),-1
3,Analytics Manager,$105K-$167K (Glassdoor est.),Responsible for both high level and granular m...,4.4,OneMagnify\n4.4,"Dearborn, MI","Detroit, MI",201 to 500 employees,1967,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1
4,Computational Chemist/Data Scientist,$105K-$167K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310562\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,Data Engineer/Sr Data Engineer - Data Engineer...,$106K-$123K (Glassdoor est.),Data Engineer/Sr Data Engineer - Data Engineer...,3.3,Liberty Mutual Insurance\n3.3,"Seattle, WA","Boston, MA",10000+ employees,1912,Company - Private,Insurance Carriers,Insurance,$10+ billion (USD),"Travelers, Allstate, State Farm"
401,Data Engineer,$106K-$123K (Glassdoor est.),Your Opportunity\n\n\nDo you want to be part o...,3.6,Charles Schwab\n3.6,"Westlake, TX","San Francisco, CA",10000+ employees,1973,Company - Public,Investment Banking & Asset Management,Finance,$5 to $10 billion (USD),-1
402,Senior WAN Engineer,$106K-$123K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nLocat...",4.1,ManTech\n4.1,"Williston, VT","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
403,"CQV Engineers - Compliance, Cleaning Validatio...",$106K-$123K (Glassdoor est.),CAI is a 100% employee-owned company establish...,4.3,CAI\n4.3,"Denver, CO","Indianapolis, IN",501 to 1000 employees,1996,Company - Private,Architectural & Engineering Services,Business Services,$50 to $100 million (USD),"Sequence, PSC Biotech Corporation, ProPharma G..."


In [128]:
df72 = get_jobs(5, False, path, 10, 2)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [129]:
df73 = get_jobs(5, False, path, 10, 3)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [130]:
df74 = get_jobs(5, False, path, 10, 4)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [131]:
df75 = get_jobs(5, False, path, 10, 5)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [132]:
df76 = get_jobs(5, False, path, 10, 6)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [133]:
df77 = get_jobs(5, False, path, 10, 7)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [134]:
df78 = get_jobs(5, False, path, 10, 8)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [135]:
df79 = get_jobs(5, False, path, 10, 9)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [136]:
df80 = get_jobs(5, False, path, 10, 10)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [137]:
df81 = get_jobs(5, False, path, 10, 11)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [138]:
df82 = get_jobs(5, False, path, 10, 12)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [139]:
df83 = get_jobs(5, False, path, 10, 13)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [140]:
df84 = get_jobs(5, False, path, 10, 14)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [141]:
df85 = get_jobs(5, False, path, 10, 15)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [142]:
df86 = get_jobs(5, False, path, 10, 16)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [143]:
df87 = get_jobs(5, False, path, 10, 17)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [144]:
df88 = get_jobs(5, False, path, 10, 18)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [146]:
df89 = get_jobs(5, False, path, 10, 19)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [147]:
df90 = get_jobs(5, False, path, 10, 20)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [148]:
df91 = get_jobs(5, False, path, 10, 21)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [149]:
df92 = get_jobs(5, False, path, 10, 22)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [150]:
df93 = get_jobs(5, False, path, 10, 23)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [151]:
df94 = get_jobs(5, False, path, 10, 24)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [152]:
df95 = get_jobs(5, False, path, 10, 25)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [153]:
df96 = get_jobs(5, False, path, 10, 26)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [154]:
df97 = get_jobs(5, False, path, 10, 27)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [155]:
df98 = get_jobs(5, False, path, 10, 28)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [156]:
df99 = get_jobs(5, False, path, 10, 29)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [157]:
df100 = get_jobs(5, False, path, 10, 30)

 x out failed
Progress: 0/5
Progress: 1/5
Progress: 2/5
Progress: 3/5
Progress: 4/5
Progress: 5/5


In [158]:
finaldf = pd.concat([finaldf, df72,df73,df74,df75,df76,df77,df78,df79,df80,df81,df82,df83,df84,df85,df86,df87,df88,df89,df90,df91,df92,df93,df94,df95,df96,df97,df98,df99,df100], ignore_index = True)

In [159]:
finaldf

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Senior Data Scientist,$105K-$167K (Glassdoor est.),"About Us\n\nAt GutCheck, we pioneered agile ma...",3.8,GutCheck\n3.8,"Denver, CO","Denver, CO",51 to 200 employees,2009,Company - Private,Advertising & Marketing,Business Services,$10 to $25 million (USD),"Nielsen, Zappi, SurveyMonkey"
1,Data Scientist,$105K-$167K (Glassdoor est.),Data Scientist\n\nAs Data Scientist at HP Inc....,4.0,HP Inc.\n4.0,"San Diego, CA","Palo Alto, CA",10000+ employees,1939,Company - Public,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
2,"Scientist/ Sr. Engineer I, Process Development",$105K-$167K (Glassdoor est.),The Role:\nCurious about HIGH-THROUGHPUT PROCE...,3.4,Moderna Therapeutics\n3.4,"Norwood, MA","Cambridge, MA",501 to 1000 employees,2011,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$1 to $5 million (USD),-1
3,Analytics Manager,$105K-$167K (Glassdoor est.),Responsible for both high level and granular m...,4.4,OneMagnify\n4.4,"Dearborn, MI","Detroit, MI",201 to 500 employees,1967,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1
4,Computational Chemist/Data Scientist,$105K-$167K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310562\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,Sr. Software Engineer (Machine Learning),$146K-$166K (Glassdoor est.),Technical professionals are defined by what th...,3.4,"Samsung Semiconductor, Inc.\n3.4","San Jose, CA","Suwon, South Korea",1001 to 5000 employees,1983,Subsidiary or Business Segment,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
546,Principal Software Engineer for Machine Learning,$146K-$166K (Glassdoor est.),DECISIVE ANALYTICS Corporation is looking for ...,4.7,DECISIVE ANALYTICS Corporation\n4.7,"Arlington, VA","Arlington, VA",51 to 200 employees,1996,Company - Private,Aerospace & Defense,Aerospace & Defense,$25 to $50 million (USD),"Torch Technologies, American Systems"
547,Engineer - Software (Mid-Senior),$146K-$166K (Glassdoor est.),Is it time for a new opportunity? Want to work...,4.6,Space Dynamics Laboratory\n4.6,"Albuquerque, NM","Logan, UT",501 to 1000 employees,1959,Nonprofit Organization,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
548,Point72 Healthcare Data Scientist,$146K-$166K (Glassdoor est.),About Point72\nPoint72 Asset Management is a g...,3.9,Point72\n3.9,"New York, NY","Stamford, CT",1001 to 5000 employees,2014,Company - Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable,-1


In [161]:
finaldf.duplicated().sum()

4

In [162]:
finaldf.drop_duplicates(inplace = True)

In [163]:
finaldf.duplicated().sum()

0

In [164]:
finaldf.to_csv('glassdoor_jobs2.csv', index = False)